In [ ]:
#분석 베이스 데이터 로드
import pandas as pd
import numpy as np
from collections import Counter
import ast
import json

# Long 포맷 데이터 로드
df_long = pd.read_csv("mp_survey/processed/responses_long.csv")

print("📊 분석 시작!")
print(f"👥 사용자 수: {df_long['user_id'].nunique()}")
print(f"📝 총 응답 수: {len(df_long)}")
print("\n샘플 데이터:")
print(df_long.head(3))


In [ ]:
#모든 사용자 분석
def analyze_user_ego(user_id):
    """한 사용자 자아별 완전 분석"""
    df_u = df_long[df_long['user_id'] == user_id].copy()

    result = {}
    for ego in ['본능 자아', '관계 자아', '목표 자아']:
        df_e = df_u[df_u['ego'] == ego]

        if len(df_e) == 0:
            result[ego] = {"themes": {}, "avg_score": 0, "percent": 0}
            continue

        # 🔍 테마 빈도
        themes_flat = []
        for themes_str in df_e['themes']:
            try:
                themes = ast.literal_eval(themes_str)
                themes_flat.extend(themes)
            except:
                themes_flat.append(themes_str)

        theme_counts = dict(Counter(themes_flat).most_common(10))

        # 📊 평균 점수 → 탐색 정도 (%)
        avg_score = df_e['score'].mean()
        percent = round((avg_score - 1) / 4 * 100, 1)  # 1~5 → 0~100

        result[ego] = {
            "theme_counts": theme_counts,
            "avg_score": round(avg_score, 2),
            "percent": percent,
            "top_themes": [(t, c) for t, c in theme_counts.items()][:5]
        }

    return result

print("✅ 사용자 분석 함수 준비 완료!")


In [ ]:
# 모든 사용자 분석 (30명이라면 3초!)
users = df_long['user_id'].unique()
all_user_analysis = {}

for user_id in users:
    analysis = analyze_user_ego(user_id)
    all_user_analysis[user_id] = analysis

    print(f"✅ {user_id}: 분석 완료")

# JSON으로 저장
with open("mp_survey/processed/all_user_analysis.json", "w", encoding="utf-8") as f:
    json.dump(all_user_analysis, f, ensure_ascii=False, indent=2)

print(f"\n🎉 {len(users)}명 사용자 분석 완료!")
print("📁 mp_survey/processed/all_user_analysis.json 저장됨")


In [ ]:
#자아별 탐색 정도 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 첫 번째 사용자 예시로 시각화
sample_user = list(all_user_analysis.keys())[0]
user_data = all_user_analysis[sample_user]

# 도넛 차트 (탐색 정도)
fig, ax = plt.subplots(figsize=(8, 6))
egos = list(user_data.keys())
percents = [user_data[ego]['percent'] for ego in egos]

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
wedges, texts, autotexts = ax.pie(percents, labels=egos, autopct='%1.1f%%',
                                  colors=colors, startangle=90, pctdistance=0.85)

# 도넛 모양
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig.gca().add_artist(centre_circle)

plt.title(f"{sample_user} - 자아별 탐색 정도", fontsize=16, pad=20)
plt.tight_layout()
plt.show()

print("📊 첫 번째 사용자 자아 탐색 정도:")
for ego, data in user_data.items():
    print(f"  {ego}: {data['percent']}% (평균 {data['avg_score']:.2f}점)")


In [ ]:
#자아별 키워드 순위 + 워드 클라우드
from wordcloud import WordCloud

def create_user_wordclouds(user_id):
    """사용자별 자아 워드클라우드 생성"""
    analysis = all_user_analysis[user_id]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    for i, ego in enumerate(['본능 자아', '관계 자아', '목표 자아']):
        counts = analysis[ego]['theme_counts']
        if not counts:
            axes[i].text(0.5, 0.5, '데이터 없음', ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(ego)
            continue

        # 워드클라우드
        wc = WordCloud(width=400, height=400, background_color='white',
                      colormap='viridis', font_path=None).generate_from_frequencies(counts)
        axes[i].imshow(wc, interpolation='bilinear')
        axes[i].set_title(f"{ego}\n({analysis[ego]['percent']}%)")
        axes[i].axis('off')

    plt.tight_layout()
    plt.savefig(f"mp_survey/users/{user_id}_wordclouds.png", dpi=150, bbox_inches='tight')
    plt.show()
    return f"mp_survey/users/{user_id}_wordclouds.png"

# 첫 3명 워드클라우드 생성
for user_id in list(all_user_analysis.keys())[:3]:
    print(f"\n🎨 {user_id} 워드클라우드 생성:")
    img_path = create_user_wordclouds(user_id)


In [ ]:
#LLM 요약 + 가이드 생성 준비 (open AI 활용)
# OpenAI API 키 입력 (비밀)
import openai
# openai.api_key = "your-api-key-here"

def llm_summarize_user(user_id):
    """LLM으로 자아 요약 + 가이드 생성"""
    analysis = all_user_analysis[user_id]

    prompt = f"""
사용자 {user_id}의 자아 분석 결과:

"""
    for ego, data in analysis.items():
        top5 = ', '.join([t for t,c in data['top_themes']])
        prompt += f"- {ego}: {data['percent']}%, 주요 키워드: [{top5}]\n"

    prompt += """
위 결과를 바탕으로:
1. 각 자아 1줄씩 요약 (총 3줄)
2. 탐색 정도 낮은 자아 중심 가이드 (2가지 연습)
3. 세 자아 균형 가이드 (2~3줄)

따뜻하고 중립적인 톤으로 작성해줘.
"""

    # response = openai.ChatCompletion.create(
    #     model="gpt-3.5-turbo",
    #     messages=[{"role": "user", "content": prompt}]
    # )

    # LLM 응답 파싱해서 JSON 저장 (구현시)
    llm_result = {
        "summary": {ego: f"{ego} 요약 예정" for ego in analysis.keys()},
        "exploration_guide": "탐색 가이드 예정",
        "balance_guide": "균형 가이드 예정"
    }

    # 사용자별 LLM 결과 저장
    with open(f"mp_survey/users/{user_id}_llm_summary.json", "w", encoding="utf-8") as f:
        json.dump(llm_result, f, ensure_ascii=False, indent=2)

    return llm_result

# 테스트 (첫 사용자)
print("🤖 LLM 요약 테스트:")
llm_result = llm_summarize_user(list(all_user_analysis.keys())[0])
print(json.dumps(llm_result, ensure_ascii=False, indent=2))


In [ ]:
# 🎉 모든 사용자 결과 자동 생성!
results_summary = []

for user_id in list(all_user_analysis.keys()):
    analysis = all_user_analysis[user_id]

    # 워드클라우드 생성
    create_user_wordclouds(user_id)

    # LLM 요약 (테스트용)
    llm_result = llm_summarize_user(user_id)

    # 최종 사용자 리포트 JSON
    user_report = {
        "user_id": user_id,
        "ego_overview": analysis,
        "wordcloud_path": f"mp_survey/users/{user_id}_wordclouds.png",
        "llm_summary": llm_result
    }

    # 사용자별 JSON 저장
    with open(f"mp_survey/users/{user_id}_report.json", "w", encoding="utf-8") as f:
        json.dump(user_report, f, ensure_ascii=False, indent=2)

    results_summary.append({
        "user_id": user_id,
        "본능": analysis['본능 자아']['percent'],
        "관계": analysis['관계 자아']['percent'],
        "목표": analysis['목표 자아']['percent']
    })

    print(f"✅ {user_id} 완료")

# 전체 요약 CSV
pd.DataFrame(results_summary).to_csv("mp_survey/processed/user_summary.csv", index=False)
print("\n🎊 전체 분석 완료!")
print("📁 mp_survey/users/ 폴더에 사용자별 결과 저장됨")
